<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex11notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex11notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

---
## 判別分析 (1)
---



In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# 文字列を Markdown 形式で（数式は LaTeX でフォーマットして）表示
from IPython.display import display, Markdown

# SciPy
import scipy as sp
# SciPy の 統計関数群の中の正規分布モジュール (scipy.stats.norm) と多変量正規分布モジュール (scipy.stats.multivariate_normal)
from scipy.stats import norm, multivariate_normal

---
### 判別分析とは



#### 例： (身長, 体重) から「人間」か「ほげ星人」かを判別する

「人間」と「ほげ星人」あわせて200人分の $(\mbox{身長[cm]}, \mbox{体重[kg]})$ の数値を集めたデータがあったとします．それぞれのデータには，人間の数値なのかほげ星人の数値なのかを表す変数も含まれているとします（注）．
以下のセルの出力の `Height` 列は身長，`Weight` 列は体重を表します．`Class` 列の値は，人間を表す `Human` とほげ星人を表す `Hoge` のいずれかです．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: これは，説明を理解してもらいやすくするために作った嘘のデータです．このデータ中のほげ星人に近い身長・体重のひとがいたとしても，そのひとが人間ではないというようなことを主張するものではありません．
</span>

In [ ]:
# 人間 vs ほげ星人
URL = 'https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/humanvshoge.csv'
dfHoge = pd.read_csv(URL)
dfHoge

`Class` の値で色分けして (`Height`, `Weight`) の散布図を描いてみると，次のようになっています．

In [ ]:
# 「人間 vs ほげ星人」の散布図
fig, ax = plt.subplots(figsize=(8, 6))
dfTmp = dfHoge.loc[dfHoge['Class'] == 'Human']
ax.scatter(dfTmp['Height'], dfTmp['Weight'], label='Human')
dfTmp = dfHoge.loc[dfHoge['Class'] == 'Hoge']
ax.scatter(dfTmp['Height'], dfTmp['Weight'], label='Hoge')
xmin, xmax = 0, 250
ymin, ymax = 0, 150
ax.set_xlim(xmin, xmax)
ax.set_xlabel('Height')
ax.set_ylim(ymin, ymax)
ax.set_ylabel('Weight')
ax.set_aspect('equal')
ax.legend()
plt.show()

このようなデータが与えられたときに，「人間かほげ星人か分からない（`Class` の値が未知な）ひとの (`Height`, `Width`) の値から，そのひとの `Class` を決定したい」というのが，判別分析の問題の一つの例です．

#### 問題設定

上記の例では，個々のデータが「人間」と「ほげ星人」のどちらのグループに属するのか決めることを考えていました．判別分析では，これらのグループのことを，**クラス** (class) や **カテゴリ** (category)，**群** (group) と呼びます．この例では「人間」クラスと「ほげ星人」クラスの二つを考えていますが，クラスの数は3つ以上あっても構いません．

判別分析でやりたいことを一般化して説明すると，次のようになります：

> 多変量のデータがあり，その個々のデータは複数のクラスのうちのいずれかに所属しているとする．これらのデータを基にして，新しいデータがどのクラスに所属するのかを決定する仕組み（これを **判別関数** といいます）を作りたい．

統計学ではこのような問題を **判別分析** (discriminant analysis) と呼びますが，機械学習等の分野では，**分類** (classification) や **識別** (discrimination)，あるいは **パターン認識** (pattern recognition) と呼びます（「機械学習I/II」の授業で出てきます）．


これまで扱ってきた多変量解析手法では，変数はすべて量的な変数でしたが，判別分析では，説明変数が量的な変数で，被説明変数は質的な変数（注）となっているとみなせます．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※ 注意: より詳しく言うと，「人間」と「ほげ星人」みたいに少数の値をとる，「カテゴリカル」な変数．
</span>


#### 例： Fisher のアヤメのデータ

判別分析の問題の例をもう一つあげます．植物の [アヤメ](https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%A4%E3%83%A1) の種を分類する問題です．

In [ ]:
# Fisher のアヤメのデータ
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
dfIris = iris.frame
dfIris['species'] = 'hoge'
for k, tn in enumerate(iris.target_names):
    dfIris.loc[dfIris['target'] == k, 'species'] = tn
dfIris.drop(columns='target', inplace=True)
dfIris

このデータは，

- Iris setosa: [Wikipedia](https://en.wikipedia.org/wiki/Iris_setosa)，[Wikipedia日本語版 ヒオウギアヤメ]( https://ja.wikipedia.org/wiki/%E3%83%92%E3%82%AA%E3%82%A6%E3%82%AE%E3%82%A2%E3%83%A4%E3%83%A1)
- Iris versicolor [Wikipedia](https://en.wikipedia.org/wiki/Iris_versicolor)
- Iris virginica [Wikipedia](https://en.wikipedia.org/wiki/Iris_virginica)

という3種のアヤメの`sepal length`（がく片の長さ[cm]），`sepal width`（がく片の幅[cm]），`petal length`（花弁の長さ[cm]），`petal width`（花弁の幅[cm]）の数値から成るものです．

4つの数値が組になったデータですので，先の例のようにそのまま散布図を描くことはできませんが，4つのうち任意の2つの変数を選んで散布図を描くと，次のようになっています．

In [ ]:
# Seaborn のペアプロット関数（多次元データの変数間の散布図等を自動的に描いてくれる）
seaborn.pairplot(dfIris, hue='species', corner=True)

このデータに対する判別分析の問題は，
何らかの方法で，(`sepal length`, `sepal width` , `petal length`, `petal width`) という4つの数値から，この数値の組を持つアヤメの `species` （setosa / versicolor / virginica のいずれか）を予測する，ということになります．

余談ですが，このアヤメのデータは，有名な統計学者である Ronald Fisher が判別分析の研究に用いたことから，「Fisher's Iris data set」として知られています．多変量解析や機械学習・パターン認識の教科書等によく出てきます．ここでは，scikit-learn という機械学習ライブラリで提供されているものを利用しています．

- Ronald Fisher （ロナルド・フィッシャー）:  イギリスの統計学者・進化生物学者， 1890 - 1962， [Wikipedia](https://ja.wikipedia.org/wiki/%E3%83%AD%E3%83%8A%E3%83%AB%E3%83%89%E3%83%BB%E3%83%95%E3%82%A3%E3%83%83%E3%82%B7%E3%83%A3%E3%83%BC)
- [sklearn.datasets.load_iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html)

---
### 2クラス1次元の判別分析

ここでは，データは1次元，つまり1つの変数のみから成るものとし，さらに，クラスの数を 2 に限定して，判別分析の考え方や具体的な方法を説明します．

#### 考え方




判別分析では，データの背後に正規分布を仮定します．つまり，個々のデータは，何らかの決まったパラメータを持った正規分布から抽出された標本だとします．その正規分布は，クラスごとに一つずつ考えます．例えば，1次元のデータを「クラス1」と「クラス2」の2クラスに分類する問題の場合，

- クラス1の正規分布 $N(\mu_1, \sigma_1^2)$
- クラス2の正規分布 $N(\mu_2, \sigma_2^2)$

という2つを考えます．これらの正規分布のパラメータは，与えられたデータから適当な方法で推定します．

このとき，「$x$ という値をもつデータがクラス1の正規分布から得られたとすることの尤度」と，「$x$ という値をもつデータがクラス2の正規分布から得られたとすることの尤度」を求めてやれば，「$x$ という値をもつデータは，尤度のより大きい方の正規分布に対応するクラスに属すと判定する」ことができるでしょう．これが，判別分析の基本的な考え方です．

上記の2つの正規分布の場合，尤度はぞれぞれ
$$
\begin{aligned}
\ell_1(x) &= \frac{1}{\sqrt{2\pi\sigma_1^2}} \exp{\left( - \frac{(x-\mu_1)^2}{2\sigma_1^2}\right)} \qquad (1)\\
\ell_2(x) &= \frac{1}{\sqrt{2\pi\sigma_2^2}} \exp{\left( - \frac{(x-\mu_2)^2}{2\sigma_2^2}\right)} \qquad (2)\\
\end{aligned}
$$
と表せます．この $\ell_1(x)$ と $\ell_2(x)$ の値を求めて，「$x$という値をもつデータは，$\ell_1(x) > \ell_2(x)$ ならクラス1に，$\ell_1(x) < \ell_2(x)$ ならクラス2に属すと判定する」ことになります．

下図において，青い方がクラス1の，オレンジの方がクラス2の正規分布の確率密度関数だったとすると，次のようになります．

- $\ell_1(x_1) > \ell_2(x_1)$ なので，$x = x_1$ のデータはクラス1に分類
- $\ell_1(x_2) < \ell_2(x_2)$ なので，$x = x_2$ のデータはクラス2に分類




<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/likelihood.png">

#### 判別関数

上記で説明していることをまとめると，クラス1,2のどちらに所属するかが未知のデータ $x$ が与えられたときにその所属クラスを判定する手続きは，次のようになります．


1. 「$x$ という値をもつデータがクラス1の正規分布から得られたとすることの尤度」 $\ell_1(x)$ および「クラス2の正規分布から得られたとすることの尤度」$\ell_2(x)$ の値を計算する
1. $\ell_1(x), \ell_2(x)$ の大小によって所属クラスを判定する（大きい方に対応するクラスに所属すると判定する）

この説明では尤度を用いることになっていますが，実用上は，尤度そのものではなく，その対数をとったもの（対数尤度）の方が計算しやすいため，対数尤度が用いられます．この場合，

$$
h(x) = \log{\ell_1(x)} - \log{\ell_2(x)} \qquad (3)
$$

という関数を考えて，この値の正負によってクラス1,2の判別をします（ $h(x) > 0$ ならクラス1， $h(x) < 0$ ならクラス2）．この関数を **判別関数** (discriminant function) といいます．



「考え方」の説明で使った記号を用いて判別関数を具体的に書くと，次のようになります（注）．

$$
\begin{aligned}
h(x) &=
-\frac{1}{2}\log{2\pi\sigma_1^2} - \frac{(x-\mu_1)^2}{2\sigma_1^2}
+\frac{1}{2}\log{2\pi\sigma_2^2} + \frac{(x-\mu_2)^2}{2\sigma_2^2} \qquad (4)\\
&= - \frac{(x-\mu_1)^2}{2\sigma_1^2} + \frac{(x-\mu_2)^2}{2\sigma_2^2} + \frac{1}{2}\log{\frac{\sigma_2^2}{\sigma_1^2}} \qquad (5)
\end{aligned}
$$

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※ 注意: 詳しくいうと，これは「二次判別」と呼ばれる手法における判別関数です．「二次判別」については（それを簡略化した「線形判別」と合わせて）次回説明します．
</span>

#### 「人間」vs「ほげ星人」の判別分析（1次元）

実際の例として，「人間」vs「ほげ星人」で，身長の数値のみから人間かほげ星人か判別する問題を考えてみましょう．

まず，「人間」の身長の平均と分散 $\mu_1, \sigma_1^2$ および，「ほげ星人」の身長の平均と分散 $\mu_2, \sigma_2^2$ を推定します（ここでは標本平均，標本分散を用いています）．



In [ ]:
# X_Human が「人間」の身長，X_Hoge が「ほげ星人」の身長
X_Human = dfHoge.loc[dfHoge['Class'] == 'Human', 'Height'].to_numpy()
X_Hoge  = dfHoge.loc[dfHoge['Class'] == 'Hoge',  'Height'].to_numpy()

# それぞれの平均と分散
mu1 = np.mean(X_Human)
s1 = np.var(X_Human)
print(f'mu1 = {mu1:.2f}, s1 = {s1:.2f}')
mu2 = np.mean(X_Hoge)
s2 = np.var(X_Hoge)
print(f'mu2 = {mu2:.2f}, s2 = {s2:.2f}')

これらの値を用いて式$(5)$の判別関数の値を計算する関数を定義しておきます．

In [ ]:
# 判別関数の定義（ここでは scipy.stats.norm.logpdf を利用）
def h(x, mu1, s1, mu2, s2):
    return norm.logpdf(x, loc=mu1, scale=np.sqrt(s1)) - norm.logpdf(x, loc=mu2, scale=np.sqrt(s2))

身長のヒストグラムに $N(\mu_1, \sigma^2)$ と $N(\mu_2, \sigma^2)$ それぞれの確率密度関数を重ねて描くと，こんなんです．



In [ ]:
### 身長のヒストグラム，推定された正規分布の確率密度関数，判別境界
#
xmin, xmax = 80, 220
bins = np.linspace(xmin, xmax, 28)
xx = np.linspace(xmin, xmax, 100)

fig, ax = plt.subplots(figsize=(6, 4))
ax.hist(X_Human, bins=bins, alpha=0.8, label='Human', density=True)
ax.hist(X_Hoge,  bins=bins, alpha=0.8, label='Hoge',  density=True)

# 人間クラスの正規分布
px1 = norm.pdf(xx, loc=mu1, scale=np.sqrt(s1))
ax.plot(xx, px1, linewidth=2, color='blue')

# ほげ星人クラスの正規分布
px2 = norm.pdf(xx, loc=mu2, scale=np.sqrt(s2))
ax.plot(xx, px2, linewidth=2, color='red')

# 判別境界（ここでは数値的に求めている）
hx = lambda x: h(x, mu1, s1, mu2, s2)
p = sp.optimize.brentq(hx, xmin, xmax)
ax.axvline(p, linewidth=2, color='purple', label=f'h(x) = 0')
print(f'h(x) = 0 となる x の値は {p:.1f}')

ax.set_xlim(xmin, xmax)
ax.set_xlabel('Height')
ax.legend()
plt.show()

紫の直線は，判別関数 $h(x)$ の値が $0$ となる $x$ の値を表します．$h(x)$ によって人間クラスとほげ星人クラスを判別するときに，この点がちょうど2クラスの境界となっています．
この場合，

- 身長が 160cm のひとは，$h(160) > 0$ なので，人間と予測される
- 身長が 140cm のひとは，$h(140) < 0$ なので，ほげ星人と予測される

ことになります．

与えられたデータのそれぞれについて判別関数の値を計算し，予測が正しいかどうか調べてみましょう．

In [ ]:
# X_Human[n] の判別関数の値を求め H_Human[n] に代入
H_Human = h(X_Human, mu1, s1, mu2, s2)
# X_Hoge[n] の判別関数の値を求め H_Human[n] に代入
H_Hoge  = h(X_Hoge,  mu1, s1, mu2, s2)

# 混同行列 confusion[i, j] は，正解が i 番目のクラスで予測が j 番目のクラスだったデータの数
confusion = np.empty((2, 2), dtype=int)
c = np.sum(H_Human >= 0)
confusion[0, :] = [c, len(X_Human) - c]
c = np.sum(H_Hoge >= 0)
confusion[1, :] = [c, len(X_Hoge) - c]

# 混同行列を Markdown の表形式で表示
ss = f'''
| |予測が Human|予測が Hoge|
|:--|--:|--:|
|**正解が Human**|{confusion[0, 0]}|{confusion[0, 1]}|
|**正解が Hoge**|{confusion[1, 0]}|{confusion[1, 1]}|
'''
display(Markdown(ss))

人間については100人中98人を正しく人間と判別し，ほげ星人については100人中99人を正しくほげ星人と判別しています．
間違っているところは，上のヒストグラムで判別境界の反対側にはみ出している部分です．

次に，体重の方のデータで同じことをやってみましょう．

In [ ]:
# X_Human が「人間」の体重，X_Hoge が「ほげ星人」の体重
X_Human = dfHoge.loc[dfHoge['Class'] == 'Human', 'Weight'].to_numpy()
X_Hoge  = dfHoge.loc[dfHoge['Class'] == 'Hoge',  'Weight'].to_numpy()

# それぞれの平均と分散
mu1 = np.mean(X_Human)
s1 = np.var(X_Human)
print(f'mu1 = {mu1:.2f}, s1 = {s1:.2f}')
mu2 = np.mean(X_Hoge)
s2 = np.var(X_Hoge)
print(f'mu2 = {mu2:.2f}, s2 = {s2:.2f}')

In [ ]:
### 体重のヒストグラム，推定された正規分布の確率密度関数，判別境界
#
xmin, xmax = 0, 150
bins = np.linspace(xmin, xmax, 28)
xx = np.linspace(xmin, xmax, 100)

fig, ax = plt.subplots(figsize=(6, 4))
ax.hist(X_Human, bins=bins, alpha=0.8, label='Human', density=True)
ax.hist(X_Hoge,  bins=bins, alpha=0.8, label='Hoge',  density=True)

# 人間クラスの正規分布
px1 = norm.pdf(xx, loc=mu1, scale=np.sqrt(s1))
ax.plot(xx, px1, linewidth=2, color='blue')

# ほげ星人クラスの正規分布
px2 = norm.pdf(xx, loc=mu2, scale=np.sqrt(s2))
ax.plot(xx, px2, linewidth=2, color='red')

# 判別境界（ここでは数値的に求めている）
hx = lambda x: h(x, mu1, s1, mu2, s2)
p = sp.optimize.brentq(hx, xmin, xmax)
ax.axvline(p, linewidth=2, color='purple', label=f'h(x) = 0')
print(f'h(x) = 0 となる x の値は {p:.1f}')

ax.set_xlim(xmin, xmax)
ax.set_xlabel('Height')
ax.legend()
plt.show()

クラスを予測させてみると...




In [ ]:
# X_Human[n] の判別関数の値を求め H_Human[n] に代入
H_Human = h(X_Human, mu1, s1, mu2, s2)
# X_Hoge[n] の判別関数の値を求め H_Human[n] に代入
H_Hoge  = h(X_Hoge,  mu1, s1, mu2, s2)

# 混同行列 confusion[i, j] は，正解が i 番目のクラスで予測が j 番目のクラスだったデータの数
confusion = np.empty((2, 2), dtype=int)
c = np.sum(H_Human >= 0)
confusion[0, :] = [c, len(X_Human) - c]
c = np.sum(H_Hoge >= 0)
confusion[1, :] = [c, len(X_Hoge) - c]

# 混同行列を Markdown の表形式で表示
ss = f'''
| |予測が Human|予測が Hoge|
|:--|--:|--:|
|**正解が Human**|{confusion[0, 0]}|{confusion[0, 1]}|
|**正解が Hoge**|{confusion[1, 0]}|{confusion[1, 1]}|
'''
display(Markdown(ss))

体重を変数とした場合の判別の境界は約 81 kgで，値がこれ以下なら人間，これより大きければほげ星人と予測される結果となりました．
ヒストグラムと密度関数のグラフからも明らかですが，人間とほげ星人の値が入り混じっていますので，あまりよい予測ができていません．


---
### 2クラス多次元の判別分析

前のセクションでは，判別分析の方法を，具体的な計算の仕方も含めて説明するために，最も簡単な2クラス1次元の場合に限定して考えていました．(身長,体重)という2つの変数を持つデータを例にしてはいましたが，どちらかの変数だけで判別分析を行っていました．ここでは，2つ以上の変数を持つ多次元データの判別分析について説明し（クラス数は2に限定したまま），両方の変数を使って判別分析する実験を行ってみます．

#### 考え方



多次元データの場合も1次元の場合と考え方は同じです．$D$ 次元のデータを「クラス1」と「クラス2」の2クラスに分類する問題の場合，

- クラス1の正規分布 $N\left(\pmb{\mu}_1, \Sigma_1\right)$
- クラス2の正規分布 $N\left(\pmb{\mu}_2, \Sigma_2\right)$

という2つを考えます．これらの正規分布のパラメータについては，与えられたデータから適当な方法で推定します．

このとき，「$\pmb{x}$ という値をもつデータがクラス1の正規分布から得られたとすることの尤度」 $\ell_1(\pmb{x})$ と，「$\pmb{x}$ という値をもつデータがクラス2の正規分布から得られたとすることの尤度」$\ell_2(\pmb{x})$ はそれぞれ，次式で与えられます．

$$
\begin{aligned}
\ell_1(\pmb{x}) &= \frac{1}{\sqrt{(2\pi)^D|\Sigma_1|}} \exp{\left( -\frac{1}{2} (\pmb{x}-\pmb{\mu}_1)^{\top}\Sigma_1^{-1}(\pmb{x}-\pmb{\mu}_1) \right)} & (6)\\
\ell_2(\pmb{x}) &= \frac{1}{\sqrt{(2\pi)^D|\Sigma_2|}} \exp{\left( -\frac{1}{2} (\pmb{x}-\pmb{\mu}_2)^{\top}\Sigma_2^{-1}(\pmb{x}-\pmb{\mu}_2) \right)} & (7)\\
\end{aligned}
$$

クラスが未知のデータ $\pmb{x}$ のクラスを判別したいときは，$\ell_1(\pmb{x}), \ell_2(\pmb{x})$ の値を計算して，その値が大きい方のクラスに属するとします．


#### 判別関数

多次元の場合も，1次元の場合と同様に対数尤度を用いて次のように判別関数を定義します．

$$
h(\pmb{x}) = \log{\ell_1(\pmb{x})} - \log{\ell_2(\pmb{x})} \qquad (8)
$$

ここで，対数尤度 $\log{\ell_1(\pmb{x})}, \log{\ell_2(\pmb{x})}$ は次のように与えられます．

$$
\begin{aligned}
\log{\ell_1(\pmb{x})} &= -\frac{D}{2}\log(2\pi) - \frac{1}{2}\log{|\Sigma_1|} -\frac{1}{2} (\pmb{x}-\pmb{\mu}_1)^{\top}\Sigma_1^{-1}(\pmb{x}-\pmb{\mu}_1)  & (9)\\
\log{\ell_2(\pmb{x})} &= -\frac{D}{2}\log(2\pi) - \frac{1}{2}\log{|\Sigma_2|} - \frac{1}{2} (\pmb{x}-\pmb{\mu}_2)^{\top}\Sigma_2^{-1}(\pmb{x}-\pmb{\mu}_2) & (10)
\end{aligned}
$$

したがって，判別関数 $h(\pmb{x})$ は次のような式となります（注）．

$$
h(\pmb{x}) =  -\frac{1}{2} (\pmb{x}-\pmb{\mu}_1)^{\top}\Sigma_1^{-1}(\pmb{x}-\pmb{\mu}_1) + \frac{1}{2} (\pmb{x}-\pmb{\mu}_2)^{\top}\Sigma_2^{-1}(\pmb{x}-\pmb{\mu}_2) + \frac{1}{2}\log{\frac{|\Sigma_2|}{|\Sigma_1|}} \qquad (11)
$$

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※ 注意: 詳しくいうと，これは「二次判別」と呼ばれる手法における判別関数です．「二次判別」については（それを簡略化した「線形判別」と合わせて）次回説明します．
</span>

#### 「人間」vs「ほげ星人」の判別分析（2次元）

「人間」vs「ほげ星人」の $(身長, 体重)$ データを判別分析してみましょう．



In [ ]:
# データを格納した2次元配列をつくる
X2_Human = dfHoge.loc[dfHoge['Class'] == 'Human', ['Height', 'Weight']].to_numpy()
X2_Hoge  = dfHoge.loc[dfHoge['Class'] == 'Hoge',  ['Height', 'Weight']].to_numpy()

print('人間の最初の5人')
print(X2_Human[:5, :])
print()
print('ほげ星人の最初の5人')
print(X2_Hoge[:5, :])

まず，人間の平均 `mu_Human`，分散共分散行列 `cov_Human`，ほげ星人の平均 `mu_Hoge`，分散共分散行列 `cov_Hoge` を求めます．


In [ ]:
# 人間クラスのデータの平均と分散共分散行列
mu2_Human = np.mean(X2_Human, axis=0)
print('mu2_Human = ', mu2_Human)
XX = X2_Human - mu2_Human
cov2_Human = XX.T @ XX / XX.shape[0]
print('cov2_Human = ')
print(cov2_Human)
print()

# ほげ星人クラスのデータの平均と分散共分散行列
mu2_Hoge = np.mean(X2_Hoge, axis=0)
print('mu2_Hoge = ', mu2_Hoge)
XX = X2_Hoge - mu2_Hoge
cov2_Hoge = XX.T @ XX / XX.shape[0]
print('cov2_Hoge = ')
print(cov2_Hoge)

これらの値を用いて式$(11)$の判別関数の値を計算する関数を定義しておきます．

In [ ]:
# 判別関数の定義（scipy.stats.multivariate_normal.logpdf を利用）
def h2(X, mu1, cov1, mu2, cov2):
    L1 = multivariate_normal.logpdf(X, mean=mu1, cov=cov1)
    L2 = multivariate_normal.logpdf(X, mean=mu2, cov=cov2)
    return L1 - L2

判別分析の結果を直感的に理解できるように，データの散布図に重ねて，人間クラスほげ星人クラスそれぞれの推定された正規分布の等高線を描いてみましょう．

In [ ]:
##### 判別分析の結果を可視化する
#
fig, ax = plt.subplots(figsize=(10, 5))
xmin, xmax = 50, 250
ymin, ymax = 0, 150
xx, yy = np.mgrid[xmin:xmax:0.1, ymin:ymax:0.1]

# 人間クラスの散布図と推定された正規分布の等高線
ax.scatter(X2_Human[:, 0], X2_Human[:, 1], label='Human')
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=mu2_Human, cov=cov2_Human)
ax.contour(xx, yy, zz, colors='blue')

# ほげ星人クラスの散布図と推定された正規分布の等高線
ax.scatter(X2_Hoge[:, 0], X2_Hoge[:, 1], label='Hoge')
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=mu2_Hoge, cov=cov2_Hoge)
ax.contour(xx, yy, zz, colors='red')

# 判別境界
zz = (h2(np.dstack((xx, yy)), mu2_Human, cov2_Human, mu2_Hoge, cov2_Hoge) < 0).astype(float)
ax.contourf(xx, yy, zz, cmap='bwr', alpha=0.2)

ax.set_xlim(xmin, xmax)
ax.set_xlabel('Height')
ax.set_ylim(ymin, ymax)
ax.set_ylabel('Weight')
ax.set_aspect('equal')
ax.legend()
plt.show()

薄い青色は $h(\pmb{x}) > 0$ となる（人間と判別される）領域，薄い赤色は $h(\pmb{x}) < 0$ となる（ほげ星人と判別される）領域です．両者の境界に描かれた曲線は， $h(\pmb{x}) = 0$ となる $\pmb{x}$ の値の集合です．
例えば，$(\mbox{身長},\mbox{体重}) = (150, 60)$ のひとは人間と判別され，$(150, 100)$ のひとはほげ星人と判別されることがわかります．

それでは，個々のデータのクラスを予測させて，どれくらい正解できるかカウントしてみましょう．


In [ ]:
# X2_Human[n] の判別関数の値を求め H_Human[n] に代入
H_Human = h2(X2_Human, mu2_Human, cov2_Human, mu2_Hoge, cov2_Hoge)
# X2_Hoge[n] の判別関数の値を求め H_Human[n] に代入
H_Hoge = h2(X2_Hoge, mu2_Human, cov2_Human, mu2_Hoge, cov2_Hoge)

# 混同行列 confusion[i, j] は，正解が i 番目のクラスで予測が j 番目のクラスだったデータの数
confusion = np.empty((2, 2), dtype=int)
c = np.sum(H_Human >= 0)
confusion[0, :] = [c, len(X2_Human) - c]
c = np.sum(H_Hoge >= 0)
confusion[1, :] = [c, len(X2_Hoge) - c]

# 混同行列を Markdown の表形式で表示
ss = f'''
| |予測が Human|予測が Hoge|
|:--|--:|--:|
|**正解が Human**|{confusion[0, 0]}|{confusion[0, 1]}|
|**正解が Hoge**|{confusion[1, 0]}|{confusion[1, 1]}|
'''
display(Markdown(ss))

身長のみや体重のみの判別分析では誤った判別をする場合がありましたが，身長と体重の両方を用いる2次元の判別分析では，全てのデータを正しく判別できる結果となりました．

---
### 判別関数とマハラノビス距離

**マハラノビス距離については，ex10notebookC で解説しています**

多次元正規分布を用いる判別分析における判別関数は式(11)で表されるのでした．

$$
h(\pmb{x}) =  -\frac{1}{2} (\pmb{x}-\pmb{\mu}_1)^{\top}\Sigma_1^{-1}(\pmb{x}-\pmb{\mu}_1) + \frac{1}{2} (\pmb{x}-\pmb{\mu}_2)^{\top}\Sigma_2^{-1}(\pmb{x}-\pmb{\mu}_2) + \frac{1}{2}\log{\frac{|\Sigma_2|}{|\Sigma_1|}} \qquad (11)
$$

この式の値の計算は少々面倒なので，場合によってより簡略化した判別関数を用いることがあります（詳しいことは次回説明します）．例えば，この式の第3項を無視すると，

$$
h(\pmb{x}) = -\frac{1}{2} (\pmb{x}-\pmb{\mu}_1)^{\top}\Sigma_1^{-1}(\pmb{x}-\pmb{\mu}_1) + \frac{1}{2} (\pmb{x}-\pmb{\mu}_2)^{\top}\Sigma_2^{-1}(\pmb{x}-\pmb{\mu}_2) \qquad (12)
$$

と書けます．ここで，

- クラス1の正規分布 $N\left(\pmb{\mu}_1, \Sigma_1\right)$ に対するマハラノビス距離: $d_{1}(\pmb{x})$
- クラス2の正規分布 $N\left(\pmb{\mu}_2, \Sigma_2\right)$ に対するマハラノビス距離: $d_{2}(\pmb{x})$

とすると，式(12)の判別関数は次のように表されます．

$$
h(\pmb{x}) = \frac{d_{2}^2(\pmb{x}) - d_{1}^2(\pmb{x})}{2} \qquad (13)
$$

この式の値は，2つのマハラノビス距離の大小に応じて正負が決まります．つまり，式(12)を判別関数とする場合，クラスごとに当てはめた正規分布に対して，マハラノビス距離を規準としてより近い方の正規分布が表すクラスに所属すると判断することに相当します．
